<h1>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</h1>

<h2>Submission 1:<br/>Creation of initial data frame.</h2>

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib

In [2]:
# Load the data set for parsing with BeautifulSoup
data_link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
resp = urllib.request.urlopen(data_link)

soup = BeautifulSoup(resp.read())
table = soup.table
rows = table.find_all('tr')

<h3>
    Assumptions:
    <li>The zipcode has no hyperlinks and does not need extra processing.</li>
    <br/>
    Process the loaded data and perform the following actions:
    <li>If the row does not contain 3 items, drop the row.</li>
    <li>If a column contains a hyperlink, get the text from the hyperlink.</li>
    <li>If a column does not contain a hyperlink, use the text from that column.</li>
    <li>Remove all trailing whitespace from neighborhoods.</li>
    <li>If the row boroughs is 'Not assigned', drop the row.</li>
    <li>If the row neighborhood is 'Not assigned', use the boroughs name as the neighborhood name.</li>
</h3>

In [3]:
remapped_data = []
for row in rows:
    cols = row.find_all('td')
    if len(cols) == 3:        
        zipcode = cols[0].string

        if cols[1].find('a'):
            borough = cols[1].a.string
        else:
            borough = cols[1].string

        # Skip boroughs with name 'Not assigned'
        if borough == 'Not assigned':
            continue
            
        if cols[2].find('a'):
            neighborhood = cols[2].a.string.rstrip()
        else:
            neighborhood = cols[2].string.rstrip()

        if neighborhood == 'Not assigned':
            neighborhood = borough

        remapped_data.append([zipcode, borough, neighborhood])
        
# print(remapped_data)

<h3>Since multiple neighborhoods can exist in the same zip code, a single zipcode entry needs to aggregate all of the child neighborhoods (separated by commas).  It is assumed that a zipcode will only cover one borough.</h3>

In [4]:
zipcode_map = {}

for entry in remapped_data:
    zipcode = entry[0]
    if zipcode not in zipcode_map:
        # add borough and neighborhood to the new entry
        zipcode_map[zipcode] = [entry[1], entry[2]]
    else:
        # append this entry neighborhood to the existing record's neighborhood
        existing_entry = zipcode_map[zipcode]
        existing_entry[1] += ", "
        existing_entry[1] += entry[2]

# print(duplicate_zipcode_map)

In [ ]:
# load the dataframe with the contents of the dictionary
df = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighborhood'])

i = 0
for entry in zipcode_map:
    zipcode_data = zipcode_map[entry]
    df.loc[i] = [entry, zipcode_data[0], zipcode_data[1]]
    i += 1
    
# print(df)

In [ ]:
print(df.shape)

(103, 3)


<h2>Submission 2:<br/>Assign location information.</h2>

In [ ]:
location_data_source = "https://cocl.us/Geospatial_data"

location_df = pd.read_csv(location_data_source)
# print(location_df)

toronto_neighborhoods = pd.merge(df, location_df, left_on='Postcode', right_on='Postal Code', how='inner')
# b = a(['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])
toronto_neighborhoods.drop(['Postal Code'], axis=1, inplace=True)
print(toronto_neighborhoods.head())

  Postcode           Borough                      Neighborhood   Latitude  \
0      M3A        North York                         Parkwoods  43.753259   
1      M4A        North York                  Victoria Village  43.725882   
2      M5A  Downtown Toronto         Harbourfront, Regent Park  43.654260   
3      M6A        North York  Lawrence Heights, Lawrence Manor  43.718518   
4      M7A      Queen's Park                      Queen's Park  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494  


In [ ]:
print(toronto_neighborhoods)

    Postcode           Borough  \
0        M3A        North York   
1        M4A        North York   
2        M5A  Downtown Toronto   
3        M6A        North York   
4        M7A      Queen's Park   
5        M9A         Etobicoke   
6        M1B       Scarborough   
7        M3B        North York   
8        M4B         East York   
9        M5B  Downtown Toronto   
10       M6B        North York   
11       M9B         Etobicoke   
12       M1C       Scarborough   
13       M3C        North York   
14       M4C         East York   
15       M5C  Downtown Toronto   
16       M6C              York   
17       M9C         Etobicoke   
18       M1E       Scarborough   
19       M4E      East Toronto   
20       M5E  Downtown Toronto   
21       M6E              York   
22       M1G       Scarborough   
23       M4G         East York   
24       M5G  Downtown Toronto   
25       M6G  Downtown Toronto   
26       M1H       Scarborough   
27       M2H        North York   
28       M3H  

<h2>Submission 3:<br/>Visualization of Toronto data.</h2>

In [ ]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: / 

In [ ]:
# create map of New York using latitude and longitude values
latitude = 43.753259
longitude = -79.329656
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'],
                                           toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define credentials for Foursquare

In [ ]:
CLIENT_ID = 'x' # your Foursquare ID
CLIENT_SECRET = 'x' # your Foursquare Secret
VERSION = '20191009' # Foursquare API version
radius=500
LIMIT=100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_neighborhoods['Postcode'],
                                   latitudes=toronto_neighborhoods['Latitude'],
                                   longitudes=toronto_neighborhoods['Longitude']
                                  )


In [ ]:
print(toronto_venues.head(20))
toronto_venues.groupby('Postcode').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postcode'] = toronto_venues['Postcode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Postcode').mean().reset_index()
toronto_grouped

In [ ]:
toronto_grouped.shape

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Postcode']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postcode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')